In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from yellowbrick.classifier import ConfusionMatrix
from sklearn.svm import SVC, LinearSVC
from datetime import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [10]:
df = pd.read_csv("dataset/student_monnitoring_data.csv")

In [3]:
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
df["Day"] = df["Date"].dt.strftime("%d")

#
1. Estatísticas descritivas

In [3]:
df_desc = df.describe()
df_desc.loc["IQR"] = df_desc.loc["75%"] - df_desc.loc["25%"]

#
2. Outliers

In [ ]:
for col in df.drop(columns=["Student ID", "Date", "Class Time"]):
  plt.figure(figsize=(10,4))
  sns.histplot(
    data=df, x=col, hue="Risk Level"
    ).set_title(f"Distribuição da Variável: {col}")
  plt.show()

#
3. Análise Risk Level

In [ ]:
df["Risk Level"].value_counts(normalize=True).sort_index()

In [12]:
df_encoded = df.copy()

In [27]:
labelencoder_df1 = LabelEncoder()
df_encoded.iloc[:, 1] = labelencoder_df1.fit_transform(df_encoded.iloc[:, 1]) + 1

labelencoder_df2 = LabelEncoder()
df_encoded.iloc[:, 2] = labelencoder_df2.fit_transform(df_encoded.iloc[:, 2])

labelencoder_df3 = LabelEncoder()
df_encoded.iloc[:, 3] = labelencoder_df3.fit_transform(df_encoded.iloc[:, 3])

labelencoder_df8 = LabelEncoder()
df_encoded.iloc[:, 8] = labelencoder_df8.fit_transform(df_encoded.iloc[:, 8])

In [ ]:
plt.Figure(figsize=(12, 6))
sns.heatmap(df_encoded.corr(), annot=True)
plt.show()

In [ ]:
df_encoded.corr()["Risk Level"].sort_values()

#
4. calcule e/ou visualize (em um gráfico de barras, ou como preferir) o intervalo de confiança de 90% para a média de cada uma das variáveis físico-químicas, agrupadas pelos níveis categóricos da variável resposta quality. Que conclusões são possíveis tirar destes gráficos?

In [ ]:
for col in df_encoded.drop(columns=["Student ID", "Risk Level"]):
  sns.barplot(
    data=df_encoded, x="Risk Level", y=col, errorbar=("ci", 90),
    hue="Risk Level")
  plt.show()

In [ ]:
for col in df.drop(columns=["Student ID"]):
  plt.figure(figsize=(10,4))
  sns.histplot(
    data=df_encoded, x=col, hue=df["Risk Level"]
    ).set_title(f"Distribuição da Variável: {col}")
  plt.show()

#
5.

In [33]:
previsores = df.drop(columns=["Date", "Risk Level"]).values
classe = df["Risk Level"].values

In [43]:
labelencoder1 = LabelEncoder()
previsores[:, 1] = labelencoder1.fit_transform(previsores[:, 1])

labelencoder2 = LabelEncoder()
previsores[:, 2] = labelencoder2.fit_transform(previsores[:, 2])

labelencoder3 = LabelEncoder()
previsores[:, 7] = labelencoder3.fit_transform(previsores[:, 7])

In [5]:
x_treinamento, x_teste, y_treinamento, y_test = train_test_split(
  previsores,
  classe,
  test_size=0.3,
  random_state=0
)

In [ ]:
forest = ExtraTreesClassifier()
forest.fit(x_treinamento, y_treinamento)
importancias = forest.feature_importances_
importancias

In [7]:
x_treinamento2 = x_treinamento[:, [2,3,5,6]]
x_teste2 = x_teste[:, [2,3,5,6]]

In [ ]:
svm = SVC()
svm.fit(x_treinamento2, y_treinamento)
previsoes = svm.predict(x_teste2)
taxa_acerto = accuracy_score(y_test, previsoes)
taxa_acerto

In [ ]:
v = ConfusionMatrix(SVC())
v.fit(x_treinamento2, y_treinamento)
v.score(x_teste2, y_test)
v.poof()

In [10]:
x = pd.DataFrame(zip(list(y_test), list(previsoes)), columns=["Teste", "Previsão"])

In [ ]:
x[x["Previsão"]!= x["Teste"]]

In [12]:
svm2 = LinearSVC()
svm2.fit(x_treinamento2, y_treinamento)
previsoes2 = svm.predict(x_teste2)

In [ ]:
taxa_acerto2 = accuracy_score(y_test, previsoes2)
taxa_acerto2

In [ ]:
v2 = ConfusionMatrix(LinearSVC())
v2.fit(x_treinamento2, y_treinamento)
v2.score(x_teste2, y_test)
v2.poof()